In [1]:
from requests import get
import re
import pandas as pd


# Source
I found a wikipedia page which had a table of a few Fortune 500 computer software companies. There was a link at each row, which directed me to another page where their products were listed. 


# Web scraping

In [2]:
url="https://en.wikipedia.org/wiki/List_of_Fortune_500_computer_software_and_information_companies"
response = get(url)
print(response.text[:500]) 



<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>List of Fortune 500 computer software and information companies - Wikipedia</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of


In [3]:
from bs4 import BeautifulSoup as bs

html_soup = bs(response.text, "html.parser")
type(html_soup)


bs4.BeautifulSoup

In [4]:
companies_table = html_soup.find_all("table")
companies_table


[<table class="wikitable sortable">\n<tr>\n<th>Company</th>\n<th>Type</th>\n<th>2017 ranking</th>\n</tr>\n<tr>\n<td><a href="/wiki/Microsoft" title="Microsoft">Microsoft</a></td>\n<td>Computer Software</td>\n<td>28</td>\n</tr>\n<tr>\n<td><a href="/wiki/Oracle_Corporation" title="Oracle Corporation">Oracle</a></td>\n<td>Computer Software</td>\n<td>81</td>\n</tr>\n<tr>\n<td><a href="/wiki/Salesforce.com" title="Salesforce.com">salesforce.com</a></td>\n<td>Computer Software</td>\n<td>326</td>\n</tr>\n<tr>\n<td><a href="/wiki/Activision_Blizzard" title="Activision Blizzard">Activision Blizzard</a></td>\n<td>Computer Software</td>\n<td>406</td>\n</tr>\n<tr>\n<td><a href="/wiki/Adobe_Systems" title="Adobe Systems">Adobe Systems</a></td>\n<td>Computer Software</td>\n<td>443</td>\n</tr>\n<tr>\n<td><a href="/wiki/Symantec" title="Symantec">Symantec</a></td>\n<td>Computer Software</td>\n<td>465</td>\n</tr>\n<tr>\n<td><a class="mw-redirect" href="/wiki/Apple_Inc" title="Apple Inc">Apple Inc</a></

In [5]:
company_names=[]
products=[]
company_products=pd.DataFrame(columns=['company_name','products'])
i=1
for table in companies_table:
    companies=table.find_all("a")
    for company in companies:
        pdts=''
        #print company['title'],company['href']
        #Each company name was a link to the page, which listed their products
        url="https://en.wikipedia.org"+company['href']
        response = get(url)
        html_soup = bs(response.text, "html.parser")
        side_table=html_soup.find_all("table")
        for table in side_table:
            product=table.find("th",text="Products")
            if product != None:
                pdt=product.parent
        products=pdt.find_all("a")
        if len(products)==0:
            products=pdt.find_all("td")
        for p in products:
            pdts=pdts+p.get_text()+','
        company_products.loc[i]=[company['title'],pdts]
        i=i+1
        

In [6]:
company_products


,company_name,products
1,Microsoft,"Windows,Office,Servers,Skype,Visual Studio,Dyn..."
2,Oracle Corporation,"Oracle Applications,Oracle Database,Oracle Ent..."
3,Salesforce.com,Sales Cloud\nService Cloud\nPlatform\nMarketin...
4,Activision Blizzard,"Activision,Call of Duty,Crash Bandicoot,Destin..."
5,Adobe Systems,"List of Adobe products,"
6,Symantec,"AhnLab V3 Internet Security,Avast Antivirus,AV..."
7,Apple Inc,"Mac,iMac,iMac Pro,MacBook family,Mac Mini,Mac ..."
8,Dell Technologies,"Personal computers,Servers,SAN storage,NAS sto..."
9,HP Inc.,"Personal computers,Printers,Digital press,3D p..."
10,NCR Inc.,"Point of sale,automated teller machines,"


# Cleaning the dataframe

In [7]:
company_products.isnull().values.any()

False

There were a few pages in which the HTML structure of the page was different. I tried incorporate these special cases along with my code to handle the generic cases.
I have then, removed the trailing commas and '\n' from the columns.

In [8]:
companies_products_df=pd.DataFrame(columns=['company_name','products'])
i=1
for row in company_products.itertuples():
    pdt_list=row[2]
    str_len=len(pdt_list)
    pdt_list=pdt_list.replace("\n", ",")
    if pdt_list[str_len-1]==',':
        pdt_list = pdt_list[:-1]
    companies_products_df.loc[i]=[row[1],pdt_list]
    i=i+1

In [9]:
companies_products_df

,company_name,products
1,Microsoft,"Windows,Office,Servers,Skype,Visual Studio,Dyn..."
2,Oracle Corporation,"Oracle Applications,Oracle Database,Oracle Ent..."
3,Salesforce.com,"Sales Cloud,Service Cloud,Platform,Marketing C..."
4,Activision Blizzard,"Activision,Call of Duty,Crash Bandicoot,Destin..."
5,Adobe Systems,List of Adobe products
6,Symantec,"AhnLab V3 Internet Security,Avast Antivirus,AV..."
7,Apple Inc,"Mac,iMac,iMac Pro,MacBook family,Mac Mini,Mac ..."
8,Dell Technologies,"Personal computers,Servers,SAN storage,NAS sto..."
9,HP Inc.,"Personal computers,Printers,Digital press,3D p..."
10,NCR Inc.,"Point of sale,automated teller machines"


# Converting to JSON format to write into a file- Task3.json
While I could have used functions like to_json to convert to json format, the to_json returns a string whos structure 
is a little different from the structure that we want. ( to_json function includes column names or index, and the structure/ hierarchy is a little different from what we want)
So I have treated it as s string to create a json structure, which I will later use with the json.dumps() function to put it into a JSON file.

In [10]:
json_struc='{"DATA":{'
for row in companies_products_df.itertuples():
    json_struc=json_struc+'"'+row[1]+'":'
    pdts=row[2].replace(',','","')
    pdt_array='["'+pdts+'"]'
    json_struc=json_struc+pdt_array+','
json_struc=json_struc+'}}' 
json_struc

u'{"DATA":{"Microsoft":["Windows","Office","Servers","Skype","Visual Studio","Dynamics","Xbox","Surface","Mobile","more..."],"Oracle Corporation":["Oracle Applications","Oracle Database","Oracle Enterprise Manager","Oracle Fusion Middleware","servers","workstations","storage","See Oracle products"],"Salesforce.com":["Sales Cloud","Service Cloud","Platform","Marketing Cloud","Commerce Cloud"],"Activision Blizzard":["Activision","Call of Duty","Crash Bandicoot","Destiny","Guitar Hero","Skylanders","Spyro the Dragon","Tony Hawk\'s","Blizzard Entertainment","Diablo","Hearthstone","Heroes of the Storm","Overwatch","StarCraft","Warcraft","King Digital","Candy Crush Saga"],"Adobe Systems":["List of Adobe products"],"Symantec":["AhnLab V3 Internet Security","Avast Antivirus","AVG AntiVirus","Avira Internet Security","Bitdefender","ClamWin","Clam AntiVirus","Comodo Antivirus","Comodo Internet Security","Dr. Web","ESET NOD32","F-Secure","F-PROT","Fortinet","G Data","Kaspersky Anti-Virus","Kasper

In [11]:
import simplejson as json
from simplejson.encoder import RawJSON
f = open("Products.json", 'w')
f.write(json.dumps(RawJSON(json_struc),indent=4))
f.close()